In [23]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import copy
import os
import time
from os import path as osp

import numpy as np
import sacred
import torch
import yaml
from sacred import Experiment
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from tracktor.config import get_output_dir
from tracktor.datasets.factory import Datasets
from tracktor.datasets.custom_wrapper import CustomSequence
from tracktor.frcnn_fpn import FRCNN_FPN
from tracktor.oracle_tracker import OracleTracker
from tracktor.reid.resnet import ReIDNetwork_resnet50
from tracktor.tracker import Tracker
from tracktor.utils import (evaluate_mot_accums, get_mot_accum,
                            interpolate_tracks, plot_sequence)

In [25]:
import cv2

In [26]:
frame_split = [0.0, 1.0]

In [27]:
obj_detects = []
obj_detect_model = "../../output/custom/faster_rcnn_fpn/training/model_epoch_30.model"
obj_detect = FRCNN_FPN(num_classes=31)
obj_detect.load_state_dict(torch.load(obj_detect_model, map_location=lambda storage, loc: storage))
obj_detects.append(obj_detect)
obj_detect.eval()
obj_detect.cuda();

In [28]:
tracker_conf = {
  "detection_person_thresh": 0.4,
  "regression_person_thresh": 0.1,
  "detection_nms_thresh": 0.2,
  "regression_nms_thresh": 0.1,
  "motion_model" : {
    "enabled": True,
    "n_steps": 5,
    "center_only": True
  },
  "public_detections": False,
  "do_align": True,
  "warp_mode": "MOTION_EUCLIDEAN",
  "number_of_iterations": 100,
  "termination_eps": 0.00001,
  "do_reid": False,
  "inactive_patience": 50,
  "max_features_num": 10,
  "reid_sim_threshold": 2.0,
  "reid_iou_threshold": 0.2
}
tracker = Tracker(obj_detect, [], tracker_conf)
tracker.reset()
num_frames = 0

In [33]:
#name = "strandvejen_faxe"
name = "bækvej_faxe"
result_dir = "../../output/custom/results"
seq = CustomSequence(name, result_dir)
data_loader = DataLoader(seq, batch_size=1, shuffle=False)

In [34]:
results = seq.load_results()
count = 0
for i, frame in enumerate(tqdm(data_loader, leave=True, position=0)):
    count += 1
    if len(seq) * frame_split[0] <= i <= len(seq) * frame_split[1]:
        with torch.no_grad():
            tracker.step(frame)
        num_frames += 1
    if count > 200: pass

results = tracker.get_results()
results = interpolate_tracks(results)
seq.write_results(results)

100%|██████████| 2160/2160 [21:19<00:00,  1.69it/s]
